In [22]:
%pip install dgl
# %pip install dgl dglgo -f https://data.dgl.ai/wheels/repo.html
%pip install torch-geometric
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-1.12.0+cu113.html

# !pip install ogb

Note: you may need to restart the kernel to use updated packages.
Looking in links: https://data.dgl.ai/wheels/repo.html
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import dgl
import torch.nn as nn
import torch.nn.functional as F
import dgl.data
from dgl.nn import GraphConv
import numpy as np
import pandas as pd
from torchsummary import summary 

c:\Users\ricca\miniconda3\envs\prnu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
moves = pd.read_json('../json/game.json') #.astype(float)

moves

,places,edges,globals
0,"{'id': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...","{'place_1': [0, 0, 1, 2, 2, 3, 4, 4, 5, 6, 7, ...","{'player_id': 1, 'victory_points': 1, 'used_kn..."
1,"{'id': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...","{'place_1': [0, 0, 1, 2, 2, 3, 4, 4, 5, 6, 7, ...","{'player_id': 2, 'victory_points': 1, 'used_kn..."
2,"{'id': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...","{'place_1': [0, 0, 1, 2, 2, 3, 4, 4, 5, 6, 7, ...","{'player_id': 3, 'victory_points': 1, 'used_kn..."
3,"{'id': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...","{'place_1': [0, 0, 1, 2, 2, 3, 4, 4, 5, 6, 7, ...","{'player_id': 4, 'victory_points': 1, 'used_kn..."
4,"{'id': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...","{'place_1': [0, 0, 1, 2, 2, 3, 4, 4, 5, 6, 7, ...","{'player_id': 4, 'victory_points': 2, 'used_kn..."
...,...,...,...
34113,"{'id': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...","{'place_1': [0, 0, 1, 2, 2, 3, 4, 4, 5, 6, 7, ...","{'player_id': 1, 'victory_points': 9, 'used_kn..."
34114,"{'id': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...","{'place_1': [0, 0, 1, 2, 2, 3, 4, 4, 5, 6, 7, ...","{'player_id': 1, 'victory_points': 9, 'used_kn..."
34115,"{'id': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...","{'place_1': [0, 0, 1, 2, 2, 3, 4, 4, 5, 6, 7, ...","{'player_id': 2, 'victory_points': 3, 'used_kn..."
34116,"{'id': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...","{'place_1': [0, 0, 1, 2, 2, 3, 4, 4, 5, 6, 7, ...","{'player_id': 2, 'victory_points': 3, 'used_kn..."


In [3]:
players = [0,0,0,0]
for i in range(len(moves)):
    players[moves.iloc[i].globals['winner']-1] += 1
print(players)

[9758, 7499, 10909, 5952]


In [4]:
def extractInputFeaturesMove(moveIndex):
  places = moves.iloc[moveIndex].places
  edges = moves.iloc[moveIndex].edges
  u = torch.tensor(edges['place_1'])
  v = torch.tensor(edges['place_2'])
  w = torch.tensor(edges['edge_owner'])
  g = dgl.graph((torch.cat([u, v], dim=0) , torch.cat([v, u], dim=0)))
  g.edata['weight'] = torch.cat([w, w], dim=0).float()
  g.ndata['feat'] = torch.tensor(np.transpose(list(places.values()))).float()
  return g

In [6]:
extractInputFeaturesMove(101).to('cuda:0')

Graph(num_nodes=54, num_edges=144,
      ndata_schemes={'feat': Scheme(shape=(11,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float32)})

In [8]:
class Net(nn.Module):
  def __init__(self, gnnInputDim, gnnHiddenDim, gnnOutputDim, globInputDim):
    super().__init__()
    self.GNN1 = GraphConv(gnnInputDim, gnnHiddenDim)
    self.GNN2 = GraphConv(gnnHiddenDim, gnnHiddenDim)
    self.GNN3 = GraphConv(gnnHiddenDim, gnnOutputDim)

    self.GlobalLayer1 = nn.Linear(globInputDim, 16)
    self.GlobalLayer2 = nn.Linear(16, 16)
    self.GlobalLayer3 = nn.Linear(16, globInputDim)


    self.OutputLayer1 = nn.Linear(54*gnnOutputDim+globInputDim, 85)
    self.OutputLayer2 = nn.Linear(85, 4)

  def forward(self, graph, globalFeats):
    graph.ndata['feat'] = F.relu(self.GNN1(graph, graph.ndata['feat']))
    graph.ndata['feat'] = F.relu(self.GNN2(graph, graph.ndata['feat']))
    embeds = F.relu(self.GNN3(graph, graph.ndata['feat']))
    embeds = torch.flatten(embeds)
    
    globalFeats = F.relu(self.GlobalLayer1(globalFeats))
    globalFeats = F.relu(self.GlobalLayer2(globalFeats))
    globalFeats = F.relu(self.GlobalLayer3(globalFeats))


    output = F.relu(self.OutputLayer1(torch.cat([embeds, globalFeats])))
    output = self.OutputLayer2(output)
    return nn.Softmax(dim=0)(output)
    

In [9]:
model = Net(g.ndata['feat'].shape[1], 8, 3, 8).to('cuda:0')
summary(model, input_size=(g.ndata['feat'].shape[1], 8, 8))

Layer (type:depth-idx)                   Param #
├─GraphConv: 1-1                         96
├─GraphConv: 1-2                         72
├─GraphConv: 1-3                         27
├─Linear: 1-4                            144
├─Linear: 1-5                            272
├─Linear: 1-6                            136
├─Linear: 1-7                            14,535
├─Linear: 1-8                            344
Total params: 15,626
Trainable params: 15,626
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─GraphConv: 1-1                         96
├─GraphConv: 1-2                         72
├─GraphConv: 1-3                         27
├─Linear: 1-4                            144
├─Linear: 1-5                            272
├─Linear: 1-6                            136
├─Linear: 1-7                            14,535
├─Linear: 1-8                            344
Total params: 15,626
Trainable params: 15,626
Non-trainable params: 0

In [10]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [11]:
idxs = np.random.permutation([x for x in range(len(moves))])
train = idxs[:5000]
validation = idxs[:500]
test = idxs[5000:6000]

In [12]:
len(moves)

34118

In [13]:
for epoch in range(5):
  print('epoch: ', epoch+1)
  for i, idx in enumerate(train):
    g = extractInputFeaturesMove(idx).to('cuda:0')
    glob = torch.tensor(list(moves.iloc[idx].globals.values())[:-1]).float().cuda()
    labels = torch.tensor(list(moves.iloc[idx].globals.values())[-1]).cuda()-1
    optimizer.zero_grad()
    outputs = model(g, glob)
    outputs = loss(outputs, labels)
    outputs.backward()
    optimizer.step()


epoch:  1
epoch:  2
epoch:  3
epoch:  4
epoch:  5


In [15]:
correct_preds = 0
tot_preds = len(validation)
with torch.no_grad():
    for i in validation:
        g = extractInputFeaturesMove(i).to('cuda:0')
        glob = torch.tensor(list(moves.iloc[i].globals.values())[:-1]).float().cuda()
        labels = torch.tensor(list(moves.iloc[i].globals.values())[-1]).cuda()-1   
        outputs = model(g, glob)
        _, predictions = torch.max(outputs, dim=-1)
        #print(labels)
        if labels.eq(predictions):
            correct_preds +=1

print('accuracy validation: ', correct_preds/tot_preds * 100)

accuracy validation:  82.0


In [17]:
correct_preds = 0
tot_preds = len(test)
with torch.no_grad():
    for i in test:
        g = extractInputFeaturesMove(i).to('cuda:0')
        glob = torch.tensor(list(moves.iloc[i].globals.values())[:-1]).float().cuda()
        labels = torch.tensor(list(moves.iloc[i].globals.values())[-1]).cuda()-1   
        outputs = model(g, glob)
        _, predictions = torch.max(outputs, dim=-1)
        #print(labels)
        if labels.eq(predictions):
            correct_preds +=1

print('accuracy test: ', correct_preds/tot_preds * 100)

accuracy test:  81.69999999999999


In [18]:
torch.save(model.state_dict(), './model_weights.pth')

In [ ]:
# NUOVO APPROCCIO BAYESIANO BOZZA:
import pyro
from pyro.nn import PyroModule, PyroSample

class Net(PyroModule):
  def __init__(self, gnnInputDim, gnnHiddenDim, gnnOutputDim, globInputDim):
    super().__init__()

    # Definizione delle distribuzioni priori per i pesi
    self.GNN1 = PyroModule[GraphConv](gnnInputDim, gnnHiddenDim)
    self.GNN1.weight = PyroSample(prior=pyro.distributions.Normal(0, 1))

    self.GNN2 = PyroModule[GraphConv](gnnHiddenDim, gnnHiddenDim)
    self.GNN2.weight = PyroSample(prior=pyro.distributions.Normal(0, 1))

    self.GNN3 = PyroModule[GraphConv](gnnHiddenDim, gnnOutputDim)
    self.GNN3.weight = PyroSample(prior=pyro.distributions.Normal(0, 1))

    self.GlobalLayer1 = PyroModule[nn.Linear](globInputDim, 16)
    self.GlobalLayer1.weight = PyroSample(prior=pyro.distributions.Normal(0, 1))

    self.GlobalLayer2 = PyroModule[nn.Linear](16, 16)
    self.GlobalLayer2.weight = PyroSample(prior=pyro.distributions.Normal(0, 1))

    self.GlobalLayer3 = PyroModule[nn.Linear](16, globInputDim)
    self.GlobalLayer3.weight = PyroSample(prior=pyro.distributions.Normal(0, 1))

    self.OutputLayer1 = PyroModule[nn.Linear](54*gnnOutputDim+globInputDim, 85)
    self.OutputLayer1.weight = PyroSample(prior=pyro.distributions.Normal(0, 1))

    self.OutputLayer2 = PyroModule[nn.Linear](85, 4)
    self.OutputLayer2.weight = PyroSample(prior=pyro.distributions.Normal(0, 1))

def forward(self, graph, globalFeats, labels=None):
    # Definizione delle distribuzioni posteriori per i pesi
    graph.ndata['feat'] = F.relu(self.GNN1(graph, graph.ndata['feat']))
    graph.ndata['feat'] = F.relu(self.GNN2(graph, graph.ndata['feat']))
    embeds = F.relu(self.GNN3(graph, graph.ndata['feat']))
    embeds = torch.flatten(embeds)

    globalFeats = F.relu(self.GlobalLayer1(globalFeats))
    globalFeats = F.relu(self.GlobalLayer2(globalFeats))
    globalFeats = F.relu(self.GlobalLayer3(globalFeats))

    output = F.relu(self.OutputLayer1(torch.cat([embeds, globalFeats])))
    output = self.OutputLayer2(output)
    
    # Definizione della funzione di guadagno per i pesi
    for name, module in self.named_modules():
        if isinstance(module, PyroModule):
            pyro.sample(f"{name}_weight", module.weight)
    
    # Calcolo della perdita per la rete neurale Bayesiana
    loss = nn.CrossEntropyLoss()(output, labels)
    for name, module in self.named_modules():
        if isinstance(module, PyroModule):
            loss += pyro.sample(f"{name}_weight", module.weight).abs().sum()

    return nn.Softmax(dim=0)(output), loss

